In [103]:
class Objekt:
    def __init__(self, name, mas, moglichePos):
        self.name = name
        self.mas = mas               # [höhe, breite]
        self.moglichePos = moglichePos[:]  # Kopie der Domain
        self.pos = None              # Aktuell gewählte Position

    def set_position(self, pos):
        self.pos = pos

    def __repr__(self):
        return f"{self.name}: pos={self.pos}, domain={self.moglichePos}"


Hilfsfunktionen für Domains & Konflikte

In [104]:
def filter_border(o, xrange, yrange):
    """Nur Randpositionen behalten (z.B. für Eingänge und Notausgänge)"""
    o.moglichePos = [p for p in o.moglichePos if p[0] in (0, xrange-1) or p[1] in (0, yrange-1)]

def conflict(o1, o2, s=1):
    """Gibt True, falls o1 und o2 sich zu nahe sind"""
    r1, c1 = o1.pos
    h1, w1 = o1.mas
    r2, c2 = o2.pos
    h2, w2 = o2.mas
    row_sep = (r1 + h1 + s <= r2) or (r2 + h2 + s <= r1)
    col_sep = (c1 + w1 + s <= c2) or (c2 + w2 + s <= c1)
    return not (row_sep or col_sep)

def total_conflicts(o, objects):
    return sum(conflict(o, other) for other in objects if other != o)

def valid_positions(o, andere, s=1):
    """Positionen, die direkt angrenzend sind"""
    valid = []
    h1, w1 = o.mas
    h2, w2 = andere.mas
    for r1, c1 in o.moglichePos:
        r2, c2 = andere.pos
        horizontal_adj = (c1 + w1 + s == c2 or c2 + w2 + s == c1) and not (r1 + h1 - 1 < r2 or r2 + h2 - 1 < r1)
        vertical_adj   = (r1 + h1 + s == r2 or r2 + h2 + s == r1) and not (c1 + w1 - 1 < c2 or c2 + w2 - 1 < c1)
        if horizontal_adj or vertical_adj:
            valid.append((r1, c1))
    return valid

def non_adjacent_positions(o, andere, s=1):
    """Positionen, die NICHT direkt angrenzend sind"""
    return [p for p in o.moglichePos if p not in valid_positions(o, andere, s)]


Setup der Objekte & Grid

In [105]:
xrange, yrange = 10, 6
all_positions = [(x, y) for x in range(xrange) for y in range(yrange)]

berg = Objekt("Berg", [4,5], all_positions)
bar  = Objekt("Bar",  [3,2], all_positions)
eing = Objekt("Eing", [1,1], all_positions)
naus = Objekt("Naus", [1,1], all_positions)

# Randpositionen für Türen filtern
for door in [eing, naus]:
    filter_border(door, xrange, yrange)

# Optional Startpositionen zufällig wählen
import random
eing.set_position(random.choice(eing.moglichePos))
naus.set_position(random.choice(naus.moglichePos))


In [106]:
def print_objects(objects):
    for o in objects:
        print(o)

print_objects([berg, bar, eing, naus])


Berg: pos=None, domain=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
Bar: pos=None, domain=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (

werte definieren

In [107]:
eing.pos = (5,0)
naus.pos = (0,4)


Für ein Objekt O:

Gehe durch alle anderen Objekte.

Prüfe conflict(O, other) → True = Konflikt.

Summiere Konflikte → total_conflicts(O, objects).

Beispiel für bar:

Bar bei (5,0)

Andere Objekte: berg(0,0), eing(5,0), naus(0,4)

In [108]:
# Objekte
objects = [berg, bar, eing, naus]

# Startpositionen
eing.set_position((5,0))
naus.set_position((0,4))
bar.set_position((0,0))
berg.set_position((0,0))  # zunächst überschneidend


In [109]:
for o in objects:
    print(f"{o.name} total conflicts: {total_conflicts(o, objects)}")


Berg total conflicts: 2
Bar total conflicts: 1
Eing total conflicts: 0
Naus total conflicts: 1


In [110]:
total_conflicts(bar, objects)

1

In [111]:
bar.moglichePos = valid_positions(bar, berg)
print_objects(objects)

Berg: pos=(0, 0), domain=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
Bar: pos=(0, 0), domain=[(5, 0), (5, 1), (5, 2), (5, 3), (5, 4)]
Eing: pos=(5, 0), domain=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (4, 0), (4, 5), (5, 0), (5, 5), (6, 0), (6, 5), (7, 0), (7, 5), (8, 0), (8, 5), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
Naus: pos=(0, 4), domain=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (4, 0), (4, 5), (5, 0), (5, 5), (6, 0), (6, 5), (7, 0), (